# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Aprendizaje no supervisado

### Trabajo práctico entregable - Grupo 22 - FIFA female players 2023

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

Inicializamos el entorno

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

Cargamos el conjunto de datos y vemos su contenido.

In [ ]:
path = 'female_players.csv'
fifa = pd.read_csv(path)

---
# Exploración y visualización

## Tamaño del dataset

In [ ]:
print(f'La base de datos original consta de {fifa.shape[0]} registros y {fifa.shape[1]} variables.')

print(f"Tiene datos desde la versión {fifa['fifa_version'].min()} hasta la versión {fifa['fifa_version'].max()}")

print('\n\nComo nos interesan sólo las jugadoras del 2023, filtramos el dataset para quedarnos con este subconjunto.')

fifa23 = fifa[fifa['fifa_version'] == 23].copy()

print(f'La base de datos 2023 consta de {fifa23.shape[0]} registros y {fifa23.shape[1]} variables.')

fifa23

## Reducción de variables

Vamos a eliminar aquellas columnas que creemos que no serán relevantes para nuestro objetivo.

**Nota:** Algunas son eliminadas por el simple hecho de que no se cuenta con una descripción precisa de la variable y/o porque son superadas por otras variables. Éstas son: `pace`, `shooting`, `passing`, `dribbling`, `defending` y `physic`.

In [ ]:
fifa23.drop(inplace=True, columns=['player_id', 'player_url', 'fifa_version', 
            'fifa_update', 'fifa_update_date', 'long_name', 'value_eur', 
            'wage_eur', 'dob', 'league_id', 'league_name', 'league_level', 
            'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 
            'club_loaned_from', 'club_joined_date', 
            'club_contract_valid_until_year', 'nationality_id', 
            'nation_team_id', 'nation_position', 'nation_jersey_number', 
            'weight_kg', 'skill_moves', 'international_reputation', 'body_type',
            'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 
            'player_face_url', 'pace', 'shooting', 'passing', 'dribbling', 
            'defending', 'physic'])

In [ ]:
print(f'Ahora tenemos {fifa23.shape[1]} variables ({110-fifa23.shape[1]} menos).')

fifa23

## Datos faltantes

Tenemos valores faltantes en `pace`, `shooting`, `passing`, `dribbling`, `defending`, `physic` y `goalkeeping_speed`. Entre las primeras 6 variables el mapa de calor de missingno nos arroja un valor de +1.00, lo cual indica que existe una probabilidad del 100% de que cuando falte un valor en la variable que está en la fila, va a faltar en la variable que está en la columna. Por ejemplo, cuando falta en `shooting`, también falta en `pace` y así. Para el caso de la variable `goalkeeping_speed` pasa lo opuesto, ya que tiene un valor de -1.00: siempre que falte un dato en esta variable, no faltará el dato asociado a las otras variables.

In [ ]:
msno.heatmap(fifa23, fontsize=12, figsize=(6, 5))
plt.show()

Acá ejemplificamos lo que acabamos de decir.

In [ ]:
faltantes = ['pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'goalkeeping_speed']
fifa23[faltantes].iloc[45:55]

Analizando los datos faltantes, están asociados a ser o no ser arquera. Esto lleva a que podríamos diferenciar el dataset entre arqueras y no arqueras. También esto nos da una pauta de qué características responden a alguien que recorre la cancha respecto a alguien que se queda al arco.

In [ ]:
fifa23[faltantes].info()

print(f"\n\nCantidad de arqueras: {len(fifa23[fifa23['player_positions'] == 'GK'])}")

## Discriminación entre arqueras y no arqueras

## Transformación de variables

El `work_rate` de cada jugadora tiene el formato ataque/defensa, catalogando cada uno en `Low`, `Medium` y `High`. Desglosamos esta variable en 2 variables diferentes: `att_work_rate` y `def_work_rate`. Luego, eliminamos la variable original.

In [ ]:
fifa23['att_work_rate'] = [i.split('/')[0] for i in fifa23['work_rate']]
fifa23['def_work_rate'] = [i.split('/')[1] for i in fifa23['work_rate']]
fifa23.drop(columns=['work_rate'],inplace=True)

Los puntajes de las posiciones aparecen con sumas y restas. Queremos que sean sólo enteros.

In [ ]:
fifa23[['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 
       'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']]

In [ ]:
for col in ['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
            'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 
            'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']:
    new = []
    for i in fifa23[col]:
        if len(i.split('+')) == 2:
            new.append(int(i.split('+')[0])+int(i.split('+')[1]))
        elif len(i.split('-')) == 2:
            new.append(int(i.split('-')[0])-int(i.split('-')[1]))
        else:
            new.append(int(i))

    fifa23[col] = new

In [ ]:
fifa23[['ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 
       'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk']]

## Simplificación de variables

Vamos a simplificar el nombre de varias variables.

In [ ]:
fifa23.columns

In [ ]:
{'short_name', 
 'player_positions', 
'height_cm', 
'nationality_name', 
'preferred_foot', 
'weak_foot', 
'shooting', 
'passing', 
'dribbling', 
'defending', 
'physic',
'attacking_crossing', 
'attacking_finishing', 
'attacking_heading_accuracy', 
'attacking_short_passing',
'attacking_volleys', 
'skill_dribbling', 
'skill_curve', 
'skill_fk_accuracy', 
'skill_long_passing', 
'skill_ball_control', 
'movement_acceleration', 
'movement_sprint_speed', 
'movement_agility', 
'movement_reactions', 
'movement_balance', 
'power_shot_power', 
'power_jumping', 
'power_stamina', 
'power_strength', 
'power_long_shots', 
'mentality_aggression', 
'mentality_interceptions', 
'mentality_positioning', 
'mentality_vision', 
'mentality_penalties', 
'mentality_composure', 
'defending_marking_awareness', 
'defending_standing_tackle', 
'defending_sliding_tackle', 
'goalkeeping_diving', 
'goalkeeping_handling', 
'goalkeeping_kicking', 
'goalkeeping_positioning', 
'goalkeeping_reflexes', 
'goalkeeping_speed', 
'att_work_rate', 
'def_work_rate'}

In [ ]:
fifa23.rename(columns = {'short_name' : 'name', 'player_positions' : 'position', 
                         'height_cm' : 'height', 
                         'nationality_name' : 'nationality'}, inplace = True)


# ventas_clean = ventas_clean[['ID_VENDEDOR', 'INSCRIPCION', 'SUB-CATEGORIA', 
#                              'DESC_TRATAMIENTO_FISCAL', 'TRATAMIENTO_FISCAL',
#                              'TRATAMIENTO_DIFERNCIAL', 'CM04', 'AÑO', 'MES', 
#                              'DEPOSITO', 'TOTAL_VENTAS', 
#                              'PORCENTAJE_COMISION_EMPRESA', 
#                              'COMISION_EMPRESA', 'MODELO']]
# ['name', 'position', 'overall', 'potential', 'age', 'height',
# 'nationality', 'preferred_foot', 'weak_foot', 'work_rate', 'pace',
# 'shooting', 'passing', 'dribbling', 'defending', 'physic',
# 'attacking_crossing', 'attacking_finishing',
# 'attacking_heading_accuracy', 'attacking_short_passing',
# 'attacking_volleys', 'skill_dribbling', 'skill_curve',
# 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
# 'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
# 'movement_reactions', 'movement_balance', 'power_shot_power',
# 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
# 'mentality_aggression', 'mentality_interceptions',
# 'mentality_positioning', 'mentality_vision', 'mentality_penalties',
# 'mentality_composure', 'defending_marking_awareness',
# 'defending_standing_tackle', 'defending_sliding_tackle',
# 'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
# 'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed',
# 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
# 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
# 'lcb', 'cb', 'rcb', 'rb', 'gk']


fifa23